In [ ]:
################################################################################
# Loading dataset
################################################################################
import gspread
from oauth2client.client import GoogleCredentials
from google.colab import auth
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
################################################################################
# Loading word embedding model
################################################################################
from gensim.models import word2vec
from gensim.models import FastText
word2vec_model = word2vec.Word2Vec.load('/content/drive/My Drive/word2vec.model')
fasttext_model = FastText.load('/content/drive/My Drive/fasttext.model')

In [ ]:
spreadsheet = gc.open('Sentiment Tagged Words')
worksheet = spreadsheet.worksheet('Subjective Words')
all_words = worksheet.col_values(1)[1:]
all_scores = worksheet.col_values(2)[1:]

In [ ]:
################################################################################
# Generating train/test sets
################################################################################
import numpy as np
from sklearn.model_selection import train_test_split

words = []
vectors = []
scores = []
for i in range(len(all_words)):
  try:
    word = all_words[i]
    vector = fasttext_model.wv[word]
    score = all_scores[i]
    words.append(word)
    vectors.append(vector)
    scores.append(score)
  except:
    continue

X_train, X_test, y_train, y_test = train_test_split(vectors, scores, test_size=0.2, random_state=42)

In [ ]:
################################################################################
# Support Vector Machines - https://scikit-learn.org/stable/modules/svm.html
################################################################################
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
################################################################################
# Gaussian Processes - https://scikit-learn.org/stable/modules/gaussian_process.html
################################################################################
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel, random_state=0).fit(X_train, y_train)
y_pred = gpc.predict(X_test)

In [ ]:
################################################################################
# Gaussian Naive Bayes - https://scikit-learn.org/stable/modules/naive_bayes.html
################################################################################
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [ ]:
################################################################################
# CatBoost Classifier - https://catboost.ai/docs/concepts/python-reference_catboostclassifier.html
################################################################################
!pip install catboost
from catboost import CatBoostClassifier, Pool

# initialize data
model = CatBoostClassifier()

# train the model
model.fit(X_train, y_train, verbose=True, plot=True)

# make the prediction using the resulting model
y_pred = model.predict(X_test)

In [ ]:
################################################################################
# Stochastic Gradient Descent - https://scikit-learn.org/stable/modules/sgd.html
################################################################################
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
################################################################################
# AdaBoost Classifier - https://scikit-learn.org/stable/modules/ensemble.html#adaboost
################################################################################
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
################################################################################
# Gradient Boosting Classifier - https://scikit-learn.org/stable/modules/ensemble.html#gradient-tree-boosting
################################################################################
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
################################################################################
# Random Forest Classifier - https://scikit-learn.org/stable/modules/ensemble.html#forests-of-randomized-trees
################################################################################
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
################################################################################
# Decision Trees - https://scikit-learn.org/stable/modules/tree.html
################################################################################
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
################################################################################
# Nearest Neighbors - https://scikit-learn.org/stable/modules/neighbors.html
################################################################################
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)

In [ ]:
################################################################################
# Performance
################################################################################
from tabulate import tabulate
from sklearn import metrics

y_true_list = y_test
y_pred_list = y_pred

classification_report = metrics.classification_report(y_true_list, y_pred_list, digits=4, output_dict=True)
results = classification_report['macro avg']
results['accuracy'] = classification_report['accuracy']

rows = [
  ['Accuracy', results['accuracy']],
  ['Precision', results['precision']],
  ['Recall', results['recall']],
  ['F1-Score', results['f1-score']]
]

print(tabulate(rows, tablefmt='github'))

In [ ]:
clf.predict([fasttext_model.wv['']])